In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier 
from sklearn.metrics import roc_auc_score

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
data_train = pd.read_csv('/content/drive/MyDrive/flight delay/train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/flight delay/test.csv')

In [27]:
data_train = data_train.dropna()

In [28]:
for i in data_test.columns:
    data_test = data_test.fillna(data_test[i].mode()[0])

In [29]:
data_train

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [30]:
data_test

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504859,4,1,2,835.0,1341.0,UA,752,N832UA,157.0,DEN,BWI,1491,3.0,26.0,X8PSLOTIV325
1504860,12,14,7,840.0,1131.0,WN,826,N268WN,84.0,MSY,FLL,673,20.0,7.0,CJYP39AHL24R
1504861,9,12,5,1035.0,1339.0,DL,50,N986DL,97.0,IAH,ATL,689,10.0,17.0,U4XZA1J0P205
1504862,10,9,4,941.0,1106.0,WN,364,N232WN,78.0,MDW,BHM,570,2.0,5.0,CMBI03GFDX2J


# Preprocessing

In [31]:
dropXY = ['Id', 'TailNum', 'FlightNum', 'TaxiOut', 'TaxiIn']
data_train = data_train.drop(dropXY, axis = 1)
data_train['ArrTime'] = (data_train['ArrTime'] // 100) * 60 + data_train['ArrTime'] % 100
data_train['DepTime'] = (data_train['DepTime'] // 100) * 60 + data_train['DepTime'] % 100

data_test = data_test.drop(dropXY, axis = 1)
data_test['ArrTime'] = (data_test['ArrTime'] // 100) * 60 + data_test['ArrTime'] % 100
data_test['DepTime'] = (data_test['DepTime'] // 100) * 60 + data_test['DepTime'] % 100

X = data_train.drop('target', axis = 1)
Y = data_train['target']

In [32]:
X

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,AirTime,Origin,Dest,Distance
0,8,14,4,814.0,880.0,XE,54.0,FAT,SAN,314
1,5,3,6,1026.0,1216.0,XE,103.0,BNA,EWR,748
2,7,11,5,803.0,907.0,XE,125.0,CLT,IAH,913
3,12,11,4,1239.0,1358.0,WN,224.0,BWI,DEN,1491
4,5,23,5,988.0,1064.0,UA,60.0,LAX,SFO,337
...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,679.0,758.0,AA,174.0,DFW,LAX,1235
1999996,2,25,1,479.0,538.0,AA,36.0,DFW,AUS,190
1999997,9,23,2,746.0,1269.0,DL,293.0,SEA,JFK,2421
1999998,7,26,6,449.0,569.0,US,97.0,PHL,JAX,742


In [33]:
Y

0          0
1          1
2          1
3          1
4          1
          ..
1999995    0
1999996    0
1999997    0
1999998    0
1999999    0
Name: target, Length: 1955844, dtype: int64

# Train catboost

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [35]:
col = 'Month', 'DayOfWeek', 'DayofMonth', 'UniqueCarrier','Origin','Dest'

In [36]:
model=CatBoostClassifier(iterations = 1200, depth = 5, cat_features = col)

In [ ]:
model.fit(X_train,y_train)

Learning rate set to 0.186835
0:	learn: 0.6460107	total: 1.48s	remaining: 29m 39s
1:	learn: 0.6145891	total: 2.94s	remaining: 29m 18s
2:	learn: 0.5897095	total: 4.31s	remaining: 28m 41s
3:	learn: 0.5758780	total: 5.71s	remaining: 28m 26s
4:	learn: 0.5665861	total: 6.92s	remaining: 27m 33s
5:	learn: 0.5574917	total: 8.11s	remaining: 26m 54s
6:	learn: 0.5517840	total: 9.35s	remaining: 26m 33s
7:	learn: 0.5482707	total: 10.7s	remaining: 26m 29s
8:	learn: 0.5454733	total: 11.9s	remaining: 26m 16s
9:	learn: 0.5432713	total: 13.5s	remaining: 26m 43s
10:	learn: 0.5384640	total: 14.8s	remaining: 26m 37s
11:	learn: 0.5367803	total: 16.1s	remaining: 26m 37s
12:	learn: 0.5338339	total: 17.6s	remaining: 26m 43s
13:	learn: 0.5317387	total: 18.8s	remaining: 26m 31s
14:	learn: 0.5306730	total: 20.1s	remaining: 26m 25s
15:	learn: 0.5287066	total: 21.4s	remaining: 26m 26s
16:	learn: 0.5279391	total: 22.5s	remaining: 26m 6s
17:	learn: 0.5268508	total: 24s	remaining: 26m 18s
18:	learn: 0.5261011	total: 2

In [ ]:
roc_auc_score(y_test,model.predict(X_test))

In [ ]:
model.fit(X,Y)

# Result

In [ ]:
sample=pd.read_csv('sample_submission.csv')

In [ ]:
y_result=model.predict_proba(data_test)[:,1]

In [ ]:
y_result.shape

In [ ]:
sample.shape

In [ ]:
sample['target']=y_result

In [ ]:
sample.to_csv('answer', index=False)